## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lompoc,US,34.6391,-120.4579,57.2,67,100,13.80,overcast clouds
1,1,Melfi,IT,40.9965,15.6514,59.0,77,20,6.91,few clouds
2,2,Avarua,CK,-21.2078,-159.7750,78.8,94,20,3.44,light rain
3,3,Makakilo City,US,21.3469,-158.0858,71.6,68,75,5.75,broken clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,80.6,78,40,11.50,scattered clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maxmimum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maxmimum temperature you would like for your trip?90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Avarua,CK,-21.2078,-159.7750,78.80,94,20,3.44,light rain
4,4,Puerto Ayora,EC,-0.7393,-90.3518,80.60,78,40,11.50,scattered clouds
6,6,Choix,MX,26.7092,-108.3219,89.60,18,100,3.85,overcast clouds
9,9,Ambon,ID,-3.6954,128.1814,80.42,79,79,3.18,broken clouds
14,14,Lorengau,PG,-2.0226,147.2712,80.22,85,97,13.98,light rain
15,15,Vanimo,PG,-2.6741,141.3028,78.15,87,99,6.91,overcast clouds
18,18,Vacaria,BR,-28.5122,-50.9339,87.01,10,6,5.99,clear sky
23,23,Rikitea,PF,-23.1203,-134.9692,78.24,75,8,11.14,light rain
27,27,Butaritari,KI,3.0707,172.7902,80.96,76,60,8.77,light rain
32,32,Mahebourg,MU,-20.4081,57.7000,78.80,89,40,5.75,light rain


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                248
City                   248
Country                248
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()
clean_df["Hotel Name"]=""
clean_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,78.80,light rain,-21.2078,-159.7750,
4,Puerto Ayora,EC,80.60,scattered clouds,-0.7393,-90.3518,
6,Choix,MX,89.60,overcast clouds,26.7092,-108.3219,
9,Ambon,ID,80.42,broken clouds,-3.6954,128.1814,
14,Lorengau,PG,80.22,light rain,-2.0226,147.2712,
15,Vanimo,PG,78.15,overcast clouds,-2.6741,141.3028,
18,Vacaria,BR,87.01,clear sky,-28.5122,-50.9339,
23,Rikitea,PF,78.24,light rain,-23.1203,-134.9692,
27,Butaritari,KI,80.96,light rain,3.0707,172.7902,
32,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,78.80,light rain,-21.2078,-159.7750,
4,Puerto Ayora,EC,80.60,scattered clouds,-0.7393,-90.3518,
6,Choix,MX,89.60,overcast clouds,26.7092,-108.3219,
9,Ambon,ID,80.42,broken clouds,-3.6954,128.1814,
14,Lorengau,PG,80.22,light rain,-2.0226,147.2712,
15,Vanimo,PG,78.15,overcast clouds,-2.6741,141.3028,
18,Vacaria,BR,87.01,clear sky,-28.5122,-50.9339,
23,Rikitea,PF,78.24,light rain,-23.1203,-134.9692,
27,Butaritari,KI,80.96,light rain,3.0707,172.7902,
32,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,78.80,light rain,-21.2078,-159.7750,Paradise Inn
4,Puerto Ayora,EC,80.60,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Choix,MX,89.60,overcast clouds,26.7092,-108.3219,Hotel Montesclaros
9,Ambon,ID,80.42,broken clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
14,Lorengau,PG,80.22,light rain,-2.0226,147.2712,Lorengau Harbourside Hotel


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))